In [3]:
import os
import random
import shutil
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import albumentations as A
from tqdm.auto import tqdm
import cv2 # Потрібен для preprocess_for_classifier

# ---------------------------------
# 1. КОНФІГУРАЦІЯ
# ---------------------------------
print("--- Етап 1: Конфігурація 'Золотого' Тест-сету ---")

# --- Налаштування ---
BASE_DIR = 'learning'
VALID_FONT_DIR = os.path.join(BASE_DIR, 'fonts_valid') # Використовуємо валідаційні шрифти

# Створюємо 2500 * 2 (чисті + аугментовані) = 5000 зображень
NUM_BASE_IMAGES = 2500

# --- Параметри генерації ---
SCENE_SIZE = (640, 640)
FONT_SIZES = [40, 50, 60, 70, 80]
MAX_DIGITS_IN_NUMBER = 5

# --- Нові шляхи ---
FINAL_TEST_DIR = os.path.join(BASE_DIR, 'final_test_set')
IMG_DIR = os.path.join(FINAL_TEST_DIR, 'images')
LBL_DIR = os.path.join(FINAL_TEST_DIR, 'labels') # Тут будуть .txt з ЧИСЛАМИ

# ---------------------------------
# 2. ОЧИЩЕННЯ ТА СТВОРЕННЯ
# ---------------------------------
if os.path.exists(FINAL_TEST_DIR):
    print(f"Очищую стару директорію: {FINAL_TEST_DIR}...")
    shutil.rmtree(FINAL_TEST_DIR)

print("Створення нових директорій...")
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(LBL_DIR, exist_ok=True)

# ---------------------------------
# 3. ДОПОМІЖНІ ФУНКЦІЇ (з "Мега-генератора")
# ---------------------------------

def get_font_files(directory):
    try:
        files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.ttf')]
        if not files:
            raise FileNotFoundError(f"Директорія {directory} порожня.")
        return files
    except FileNotFoundError as e:
        print(f"Помилка: {e}")
        return []

def create_number_block(number_str, font_path, font_size):
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        font = ImageFont.load_default()

    bboxes = []
    images = []
    current_x = 0

    for char in number_str:
        bbox = font.getbbox(char)
        char_width = bbox[2] - bbox[0]
        char_height = bbox[3] - bbox[1]
        char_img = Image.new('L', (char_width + 4, font_size + 10), 0)
        draw = ImageDraw.Draw(char_img)
        draw.text((2 - bbox[0], 5 - bbox[1]), char, font=font, fill=255)

        # Нам потрібні bboxes лише для трансформації
        x_min = current_x + 2
        y_min = 5
        x_max = x_min + char_width
        y_max = y_min + char_height
        bboxes.append([int(char), x_min, y_min, x_max, y_max]) # [class, x1, y1, x2, y2]

        images.append(char_img)
        current_x += char_img.width + random.randint(-5, 2)

    total_width = current_x
    block_height = font_size + 10
    if total_width <= 0 or block_height <= 0:
        return None, None

    block_img = Image.new('L', (total_width, block_height), 0)
    current_x = 0
    for img in images:
        block_img.paste(img, (current_x, 0))
        current_x += img.width + random.randint(-5, 2)

    return np.array(block_img), bboxes

def get_transforms(is_augmented=False):
    if not is_augmented:
        block_transform = A.NoOp()
        scene_transform = A.NoOp()
    else:
        block_transform = A.Compose([
            A.Perspective(scale=(0.02, 0.08), pad_mode=0, p=0.7),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=(-0.2, 0.2), rotate_limit=10, p=0.8),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

        scene_transform = A.Compose([
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
            A.MotionBlur(blur_limit=(3, 15), p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.6),
            A.Blur(blur_limit=(3, 5), p=0.3),
        ])
    return block_transform, scene_transform

# ---------------------------------
# 4. ГОЛОВНИЙ ЦИКЛ ГЕНЕРАЦІЇ
# ---------------------------------
print("--- Етап 2: Запуск генерації тест-сету ---")

try:
    valid_fonts = get_font_files(VALID_FONT_DIR)

    pbar = tqdm(total=NUM_BASE_IMAGES, desc="Генерація тест-сету")

    for i in range(NUM_BASE_IMAGES):

        # 1. Генеруємо "правильну" відповідь
        number_len = random.randint(1, MAX_DIGITS_IN_NUMBER)
        number_str = "".join([str(random.randint(0, 9)) for _ in range(number_len)])
        font_path = random.choice(valid_fonts)
        font_size = random.choice(FONT_SIZES)

        base_img_name = f"img_{i:05d}" # 00001, 00002...

        # 2. Генеруємо ОБИДВІ версії (чисту та аугментовану)
        versions_to_generate = [
            {"is_augmented": False, "suffix": "clean"},
            {"is_augmented": True, "suffix": "aug"}
        ]

        for config in versions_to_generate:
            is_augmented = config["is_augmented"]
            suffix = config["suffix"]

            # --- Генерація сцени ---
            scene_img = None
            while scene_img is None: # Повторюємо, якщо генерація не вдалася
                scene_bg = np.random.randint(0, 40, SCENE_SIZE, dtype=np.uint8)
                block_img, bboxes = create_number_block(number_str, font_path, font_size)
                if block_img is None: continue

                block_transform, scene_transform = get_transforms(is_augmented)
                class_labels = [b[0] for b in bboxes]
                pascal_bboxes = [b[1:] for b in bboxes]

                try:
                    transformed = block_transform(image=block_img, bboxes=pascal_bboxes, class_labels=class_labels)
                    transformed_img = transformed['image']
                except Exception:
                    continue # Помилка аугментації

                t_h, t_w = transformed_img.shape
                max_x = SCENE_SIZE[0] - t_w
                max_y = SCENE_SIZE[1] - t_h
                if max_x <= 0 or max_y <= 0: continue # Блок завеликий

                paste_x = random.randint(0, max_x)
                paste_y = random.randint(0, max_y)

                mask = (transformed_img > 0).astype(np.uint8)
                scene_bg[paste_y:paste_y+t_h, paste_x:paste_x+t_w] = np.where(
                    mask > 0, transformed_img, scene_bg[paste_y:paste_y+t_h, paste_x:paste_x+t_w])

                scene_img = scene_transform(image=scene_bg)['image']

            # --- ЗБЕРІГАЄМО ---
            current_img_name = f"{base_img_name}_{suffix}.png"
            current_lbl_name = f"{base_img_name}_{suffix}.txt"

            # 1. Зберігаємо зображення
            img_path = os.path.join(IMG_DIR, current_img_name)
            Image.fromarray(scene_img).save(img_path)

            # 2. Зберігаємо "чисту" мітку (ПРОСТО ЧИСЛО)
            lbl_path = os.path.join(LBL_DIR, current_lbl_name)
            with open(lbl_path, 'w') as f:
                f.write(number_str)

        pbar.update(1)

    pbar.close()
    print(f"\n--- ✅ Створено {NUM_BASE_IMAGES * 2} тестових зразків у {FINAL_TEST_DIR} ---")

except Exception as e:
    print(f"\n\n--- ❌ ПОМИЛКА ПІД ЧАС ВИКОНАННЯ ---")
    print(e)

--- Етап 1: Конфігурація 'Золотого' Тест-сету ---
Створення нових директорій...
--- Етап 2: Запуск генерації тест-сету ---



Генерація Тесту (x2):  44%|█████████████████████████▎                               | 1108/2500 [05:15<06:36,  3.51it/s]
/tmp/ipykernel_363054/3095340766.py:104: UserWarning: Argument(s) 'pad_mode' are not valid for transform Perspective
  A.Perspective(scale=(0.02, 0.08), pad_mode=0, p=0.7),
/home/yesman/tfvenv/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_363054/3095340766.py:109: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),

Генерація тест-сету: 100%|██████████████████████████████████████████████████████████| 2500/2500 [04:08<00:00, 10.05it/s]


--- ✅ Створено 5000 тестових зразків у learning/final_test_set ---
